In [5]:
##To Use this prediction script, please Ensure the external file: final_hospi is downloaded on the system. 
## The file path of the same has to be updated in line 24 within the double quotes ""
## If any issues still persist to access the Assessment form, kindly connect with IT team for setting up the script on your computer.
## The form can be kept open and accessed any time a specific case has to be checked for readmission assesessment.
## The script will be updated by IT team periodically for updated model and interface ease


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tkinter as tk
from tkinter import ttk
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# Suppress warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE  # Import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Load your dataset# Update file path per your directory
final_hospi = pd.read_csv("C:/Users/praga/OneDrive/My Files/Spring 2024/ML Project/Github/final_hospi_modified.csv")

# Define input features to include in the model
input_features = [
    'discharge_disposition_Discharged to home', 'insulin_No', 
    'admission_type_Emergency', 'age_[70-80)', 'insulin_Steady', 
    'admission_source_ Emergency Room', 'age_[60-70)', 'age_[80-90)', 
    'A1Cresult_Unknown', 'age_[50-60)'
]

# Check if all input features are present in the dataset
missing_columns = set(input_features) - set(final_hospi.columns)
if missing_columns:
    raise ValueError(f"The following required columns are missing from the dataset: {missing_columns}")

# Subset the data to include only the specified columns
X = final_hospi[input_features]

# Define your target variable (ensure this column is also in your dataset)
target_variable = 'readmitted'  # Adjust if your target variable name is different
Y = final_hospi[target_variable]

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# Fitting Logistic Regression to the Training set
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

def preprocess_input(values, input_features):
    # Convert incoming dictionary to DataFrame
    df = pd.DataFrame([values])

    # Ensure all expected dummy columns are present, adding them with default values if absent
    columns_to_dummy = ['race', 'gender', 'age', 'A1Cresult', 'insulin', 'admission_type', 'discharge_disposition', 'admission_source']
    for col in columns_to_dummy:
        if col not in df.columns:
            df[col] = None  # You can adjust this value depending on what you expect for missing columns

    # Apply dummy encoding to the DataFrame
    dummies = pd.get_dummies(df, columns=columns_to_dummy)
    
    # If readmitted column is part of the input and needs to be transformed
    if 'readmitted' in dummies.columns:
        dummies['readmitted'] = dummies['readmitted'].replace({'NO': 0, '>30': 0, '<30': 1})

    # Ensure all necessary columns are in the DataFrame, adding missing columns with default 0 values
    for feature in input_features:
        if feature not in dummies.columns:
            dummies[feature] = 0

    # Select only the input features needed for the model
    return dummies[input_features]


def submit():
    try:
        # Assuming vars_dict is defined elsewhere, containing the input widgets as dictionary values
        values = {k: v.get() for k, v in vars_dict.items()}
        print("Collected values:", values)  

        # Preprocess input to match model expected format
        X_input = preprocess_input(values, input_features)
        print("Preprocessed input for prediction:", X_input)  

        # Make a prediction
        prediction = model.predict(X_input)
        print("Model prediction:", prediction)  

        # Convert prediction to text
        prediction_text = "Yes" if prediction[0] == 1 else "No"
        print("Prediction text:", prediction_text)  

        # Unpacking values for the update_treeview function
        update_treeview(
            values["Discharge Disposition:"], 
            values["Insulin Use:"], 
            values["Admission Type:"], 
            values["Age:"], 
            values["Admission Source:"], 
            prediction_text
        )
    except Exception as e:
        print("Error during prediction:", str(e))  


# Function to update the Treeview, expecting specific fields
def update_treeview(discharge_disposition, insulin_use, admission_type, age, admission_source, prediction):
    tree.insert("", 'end', values=(discharge_disposition, insulin_use, admission_type, age, admission_source, prediction))


# Create main window
root = tk.Tk()
root.title("Patient Readmission Assessment Form")
root.geometry("900x700")

# Header label inside the form
header_label = ttk.Label(root, text="Patient Readmission Assessment Form", font=('Arial', 16, 'bold'))
header_label.grid(row=0, column=0, columnspan=2, pady=10)

# Instruction note label below the header
note_label = ttk.Label(root, text="Please fill all fields marked with *", font=('Arial', 9, 'italic'))
note_label.grid(row=1, column=0, columnspan=2)

# Define fields and corresponding values if applicable
fields = [
    "Patient No:", "Race:", "Gender:", "Discharge Disposition:", 
    "Insulin Use:", "Admission Type:", "Age:", "Admission Source:"
]

field_values = {
    "Race:": ["AfricanAmerican", "Asian", "Caucasian", "Hispanic", "Other", "Unknown"],
    "Gender:": ["Male", "Female", "Other"],
    "Discharge Disposition:": ["Discharged to home", "Other"],
    "Insulin Use:": ["No", "Steady"],
    "Admission Type:": ["Emergency", "Other"],
    "Age:": ["[50-60)", "[60-70)", "[70-80)", "[80-90)", "Other"],
    "Admission Source:": ["Emergency Room", "Other"]
}

vars_dict = {}
for i, field in enumerate(fields, start=2):
    display_label = field if field == "Patient No:" else field + "*"
    label = ttk.Label(root, text=display_label, font=('Arial', 12))
    label.grid(row=i, column=0, padx=10, pady=5, sticky='w')
    
    if field == "Patient No:":
        entry = ttk.Entry(root, font=('Arial', 12))
        entry.grid(row=i, column=1, padx=10, pady=5, sticky='w')
        vars_dict[field] = entry
    else:
        combo = ttk.Combobox(root, values=field_values[field], font=('Arial', 11), state='readonly')
        combo.grid(row=i, column=1, padx=10, pady=5, sticky='w')
        vars_dict[field] = combo

# Submit button to process the input and show predictions
submit_button = ttk.Button(root, text="Submit", command=lambda: submit())
submit_button.grid(row=len(fields)+2, column=0, padx=10, pady=10, columnspan=2)

# Treeview for displaying input and predictions
tree = ttk.Treeview(root, columns=("discharge_disposition", "insulin_use", "admission_type", 
                                   "age_bracket", "admission_source", "Prediction"), show="headings", height=5)
tree.heading("discharge_disposition", text="Discharge Disposition")
tree.heading("insulin_use", text="Insulin Use")
tree.heading("admission_type", text="Admission Type")
tree.heading("age_bracket", text="Age Bracket")
tree.heading("admission_source", text="Admission Source")
tree.heading("Prediction", text="Prediction")

for col in tree["columns"]:
    tree.column(col, width=120, anchor="center")

tree.grid(row=len(fields)+3, column=0, columnspan=2, padx=10, pady=20, sticky='nsew')

# Start the GUI
root.mainloop()


Collected values: {'Patient No:': '0909', 'Race:': 'Asian', 'Gender:': 'Female', 'Discharge Disposition:': 'Discharged to home', 'Insulin Use:': 'Steady', 'Admission Type:': 'Emergency', 'Age:': '[80-90)', 'Admission Source:': 'Emergency Room'}
Preprocessed input for prediction:    discharge_disposition_Discharged to home  insulin_No  \
0                                         0           0   

   admission_type_Emergency  age_[70-80)  insulin_Steady  \
0                         0            0               0   

   admission_source_ Emergency Room  age_[60-70)  age_[80-90)  \
0                                 0            0            0   

   A1Cresult_Unknown  age_[50-60)  
0                  0            0  
Model prediction: [0]
Prediction text: No
